In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from os import makedirs
from os.path import join, exists, expanduser
from keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.imagenet_utils import decode_predictions

In [2]:
img_des = pd.read_csv('D:\RMBI_MMKG_data\img_des.csv')
img_des

,LabelName,name,description
0,/m/027rn,Dominican Republic,The Dominican Republic is a country located on...
1,/m/017dcd,Mighty Morphin Power Rangers,Mighty Morphin Power Rangers is an American su...
2,/m/01sl1q,Michelle Rodriguez,Mayte Michelle Rodriguez is an American actres...
3,/m/0cnk2q,Australia national football team,The Australia men's national soccer team repre...
4,/m/02_j1w,Defender,"In the sport of association football, a defend..."
...,...,...,...
14179,/m/0dlj8q2,Le Monde's 100 Books of the Century,The 100 Books of the Century is a list of the ...
14180,/m/05zp8,Palace,"A palace is a grand residence, especially a ro..."
14181,/m/024030,Padma Vibhushan,The Padma Vibhushan is the second-highest civi...
14182,/m/02y_9cf,Ethiopian Empire,"The Ethiopian Empire, also formerly known by t..."


In [3]:
text_vec = pd.DataFrame(columns=['LabelName','name','title_vec','des_vec'])
text_vec['LabelName'] = img_des['LabelName']
text_vec['name'] = img_des['name']
text_vec['title_vec']=text_vec['title_vec'].astype(object)
text_vec['des_vec']=text_vec['des_vec'].astype(object)
text_vec

,LabelName,name,title_vec,des_vec
0,/m/027rn,Dominican Republic,NaN,NaN
1,/m/017dcd,Mighty Morphin Power Rangers,NaN,NaN
2,/m/01sl1q,Michelle Rodriguez,NaN,NaN
3,/m/0cnk2q,Australia national football team,NaN,NaN
4,/m/02_j1w,Defender,NaN,NaN
...,...,...,...,...
14179,/m/0dlj8q2,Le Monde's 100 Books of the Century,NaN,NaN
14180,/m/05zp8,Palace,NaN,NaN
14181,/m/024030,Padma Vibhushan,NaN,NaN
14182,/m/02y_9cf,Ethiopian Empire,NaN,NaN


In [4]:
import sister
sentence_embedding = sister.BertEmbedding(lang="en")

for i in range(14184):
    if (pd.notnull(img_des['name'][i])):
        text_vec['title_vec'][i] = sentence_embedding(img_des['name'][i])
    if (pd.notnull(img_des['description'][i])):
        text_vec['des_vec'][i] = sentence_embedding(img_des['description'][i]) 
text_vec

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.decoder.weight', 'predictions.decoder.bias', 'predictions.dense.bias', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\Alex\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2226: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the l

,LabelName,name,title_vec,des_vec
0,/m/027rn,Dominican Republic,"[0.4945737, 1.3592469, 1.282383, 0.01269114, -...","[0.7342588, -0.70393395, 1.6656038, 0.67461014..."
1,/m/017dcd,Mighty Morphin Power Rangers,"[1.6969184, 1.8559471, 0.8122708, -0.40268016,...","[0.025732398, -0.7696621, 2.172137, 1.2750009,..."
2,/m/01sl1q,Michelle Rodriguez,"[1.2370641, 2.8018966, 0.16598952, -0.5138393,...","[0.6779196, -0.15636724, 0.9539355, 0.6223599,..."
3,/m/0cnk2q,Australia national football team,"[-0.34985536, 1.8396395, 1.8012114, -0.3399124...","[-0.7440083, 0.010402456, 2.7207577, 0.6726733..."
4,/m/02_j1w,Defender,"[0.10609877, 1.8506274, 1.07485, -0.12472837, ...","[-0.1949073, -0.38760465, 3.056767, 0.1185953,..."
...,...,...,...,...
14179,/m/0dlj8q2,Le Monde's 100 Books of the Century,"[1.1198976, 1.1835887, -0.015412338, 0.2715615...","[0.3857916, -0.27667487, 1.3797297, 1.5173663,..."
14180,/m/05zp8,Palace,"[-0.08344516, 2.00688, 0.59392744, -0.15891127...","[0.851437, 0.03209959, 1.2786392, -0.08720866,..."
14181,/m/024030,Padma Vibhushan,"[1.5663682, 1.362892, 0.0067599267, 0.12328741...","[0.3932789, 0.06502099, 0.80699223, 0.3174034,..."
14182,/m/02y_9cf,Ethiopian Empire,"[0.6020137, 1.3195429, 0.58044046, 0.16614524,...","[-0.10884297, -1.288248, 1.3033134, 1.0716431,..."


In [6]:
y = []
x_img = []
x_title = []
x_des = []
import os
import gc

for i in range(14184):
    path_of_the_directory = os.path.join('D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader', img_des['name'][i])
    ext = ('.jpg')
    for path, dirc, files in os.walk(path_of_the_directory):
        for name in files:
            if name.endswith(ext):
                try:    
                    img_path = os.path.join(path, name)               
                    img = image.load_img(img_path, target_size=(224, 224))
                    img_arr = image.img_to_array(img)
                
                    x_img.append(img_arr)
                    y.append(img_des['name'][i])
                    x_title.append(text_vec['title_vec'][i])
                    x_des.append(text_vec['des_vec'][i])
                
                    img.close()
                    del img
                    gc.collect()
                except:
                    print(path)

C:\Users\Alex\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:819: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 2. 
  warnings.warn(str(msg))


D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\World War II
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Crystal Palace F.C.
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Crystal Palace F.C.
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\United Nations
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Marvel Entertainment
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Billy Connolly
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\APOEL FC
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Fayetteville
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\SG Dynamo Dresden
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Shrek
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Dan Povenmire
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Election
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Pittsburgh Panthers football
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Eriq La Salle
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Artist
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Gay
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Universi

C:\Users\Alex\anaconda3\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Winona Ryder
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Chris Evans
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Michael Jeter
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Roger Taylor
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\The Wrestler
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Nickelodeon
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Edmonton
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Big Love
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Coimbatore
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Saxony-Anhalt
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\London Borough of Newham
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Tony Geiss
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\ESPN
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Sandra Oh
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Tampa
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Election
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Egypt national football team
D:\RMBI_MMKG_data\Bu

In [8]:
x_img

[array([[[ 46.,  86., 156.],
         [ 48.,  88., 157.],
         [ 46.,  89., 158.],
         ...,
         [ 21.,  12.,   3.],
         [ 24.,  21.,   4.],
         [ 21.,  13.,   2.]],
 
        [[ 46.,  86., 155.],
         [ 43.,  88., 155.],
         [ 44.,  86., 158.],
         ...,
         [ 16.,   7.,   0.],
         [ 22.,  15.,   5.],
         [ 19.,  11.,   0.]],
 
        [[ 49.,  87., 158.],
         [ 45.,  90., 158.],
         [ 47.,  87., 157.],
         ...,
         [ 17.,   8.,   1.],
         [ 29.,  21.,   8.],
         [ 20.,  12.,   1.]],
 
        ...,
 
        [[154., 165., 167.],
         [152., 160., 162.],
         [150., 160., 162.],
         ...,
         [226., 209., 179.],
         [222., 211., 189.],
         [220., 204., 179.]],
 
        [[149., 160., 162.],
         [148., 160., 160.],
         [148., 158., 160.],
         ...,
         [227., 209., 187.],
         [220., 209., 187.],
         [222., 205., 187.]],
 
        [[158., 163., 159.],
 

In [ ]:
y = np.array(y)
x_img = np.array(x_img)
x_title = np.array(x_title)
x_des = np.array(x_des)

In [ ]:
print(y.shape)
print(x_img.shape)
print(x_title.shape)
print(x_des.shape)

In [ ]:
y

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
y_encoded = le.transform(y)

In [ ]:
y_encoded

In [ ]:
from tensorflow.keras.utils import to_categorical
y_one_hot = to_categorical(y_encoded)
y_one_hot

In [ ]:
y_one_hot.shape

In [ ]:
from sklearn.model_selection import train_test_split
x_img_train, x_img_test, x_title_train, x_title_test,x_des_train, x_des_test, y_one_hot_train, y_one_hot_test = train_test_split(x_img, x_title, x_des, y_one_hot ,test_size=0.2)

In [ ]:
x_img_train

In [ ]:
x_img_train, x_img_test = x_img_train / 255.0, x_img_test / 255.0

In [ ]:
x_img_train

In [ ]:
ResNet = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))
for layer in ResNet.layers:
    layer.trainable = False
ResNet = Model(inputs=ResNet.inputs, outputs=ResNet.layers[-2].output)
ResNet.summary()

In [ ]:
features = ResNet.predict(x_img_train)

In [ ]:
features_test = ResNet.predict(x_img_test)

In [ ]:
features.shape

In [ ]:
img_input = tf.keras.Input(shape=(2048,), name='img_input')
title_input = tf.keras.Input(shape=(768,), name='title_input')
des_input = tf.keras.Input(shape=(768,), name='des_input')

img_feat = tf.keras.layers.Dense(768,)(img_input)

x = tf.keras.layers.Concatenate(axis=1)([img_input,title_input,img_feat])

name_pred = tf.keras.layers.Dense(99,name="name")(x)

model = tf.keras.Model(
    inputs=[img_input, title_input, des_input],
    outputs=[name_pred],
)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(1e-3),
    loss=[
        tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    ],
    metrics=['accuracy']
)

In [ ]:
print("Fit model on training data")
history = model.fit(
    [features,x_title_train,x_des_train],
    y_one_hot_train,
    batch_size=20,
    epochs=10,
)

In [ ]:
results = model.evaluate(
    [features_test,x_title_test,x_des_test], 
    y_one_hot_test, 
    batch_size=10)